In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras import Model
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

In [18]:
data = pd.read_csv('data_preprocessed.csv', delimiter=',', encoding='latin1')

In [19]:
data

,Unnamed: 0,image_name,text_corrected,labels
0,0,image_1.jpg,look friend lightyear sohalikut trend play yea...,0
1,1,image_2.jpeg,best yearchallenge completed le year kudu nare...,0
2,2,image_3.JPG,sam thorne strippin follow follow saw everyone...,0
3,3,image_4.png,year challenge sweet dee edition,2
4,4,image_5.png,year challenge filter hilarious year challenge...,2
...,...,...,...,...
6982,6987,image_6988.jpg,tuesday mardi gras wednesday valentine friday ...,2
6983,6988,image_6989.jpg,must watch movie iti chennai meme maanagaram h...,0
6984,6989,image_6990.png,le talking planning soda junk food complaining...,1
6985,6990,image_6991.jpg,time fantasy one time unless make time arhtist...,0


In [20]:
data.drop('Unnamed: 0', axis=1,inplace=True)
data.head()

,image_name,text_corrected,labels
0,image_1.jpg,look friend lightyear sohalikut trend play yea...,0
1,image_2.jpeg,best yearchallenge completed le year kudu nare...,0
2,image_3.JPG,sam thorne strippin follow follow saw everyone...,0
3,image_4.png,year challenge sweet dee edition,2
4,image_5.png,year challenge filter hilarious year challenge...,2


In [21]:
x = data['text_corrected']
y = data['labels']

In [22]:
print(len(x),len(y))

6987 6987


In [23]:
x[:5]

0    look friend lightyear sohalikut trend play yea...
1    best yearchallenge completed le year kudu nare...
2    sam thorne strippin follow follow saw everyone...
3                    year challenge sweet dee edition 
4    year challenge filter hilarious year challenge...
Name: text_corrected, dtype: object

In [24]:
y[:5]

0    0
1    0
2    0
3    2
4    2
Name: labels, dtype: int64

In [31]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = y.reshape(-1, 1)

In [58]:
for i in range(0, len(x)): 
    if isinstance(x[i], float): x[i] = str(x[i])
        

<ipython-input-58-dd8d4e59365e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if isinstance(x[i], float): x[i] = str(x[i])


In [59]:
x_train,x_test,y_train, y_test = train_test_split(x,y)

In [60]:
print("train size: " , len(x_train),len(y_train))
print("test size: " , len(x_test),len(y_test))

train size:  5240 5240
test size:  1747 1747


In [61]:
x_train.shape

(5240,)

In [62]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [63]:
sequences[:5]

[[260, 93, 8, 106, 94, 1, 137],
 [15, 176, 8, 3, 2, 8, 261, 564],
 [10, 243, 84, 203, 236, 19, 3],
 [651, 404, 91, 26, 5, 1],
 [196]]

In [64]:
sequences_matrix

array([[  0,   0,   0, ...,  94,   1, 137],
       [  0,   0,   0, ...,   8, 261, 564],
       [  0,   0,   0, ..., 236,  19,   3],
       ...,
       [  0,   0,   0, ...,  78,   5,   1],
       [  0,   0,   0, ..., 117,  31, 184],
       [  0,   0,   0, ..., 256,  10,   2]], dtype=int32)

In [65]:
def LSTM_MODEL():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)  #vocab, vector space ;output of the Embedding layer is a 2D vector
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model